In [11]:
!pip install gradio openai sentence-transformers inference-sdk


In [12]:
!pip install openai==0.28

In [13]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# ✅ อ่านไฟล์ CSV ที่อัปโหลดไว้
rag_df = pd.read_csv("/content/dental_training_data.csv")  # เปลี่ยน path ตามที่ใช้
assert all(col in rag_df.columns for col in ["Question", "Category", "SubCategory"]), "Missing required columns"

# ✅ เตรียม embedding model
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
rag_embeddings = embedding_model.encode(rag_df["Question"].tolist(), convert_to_tensor=True)


In [14]:

def retrieve_similar_context(user_symptom: str, top_k: int = 3) -> str:
    user_embedding = embedding_model.encode(user_symptom, convert_to_tensor=True)
    hits = util.semantic_search(user_embedding, rag_embeddings, top_k=top_k)[0]
    results = []
    for hit in hits:
        idx = hit['corpus_id']
        q = rag_df.iloc[idx]["Question"]
        c = rag_df.iloc[idx]["Category"]
        s = rag_df.iloc[idx]["SubCategory"]
        results.append(f'{{"question": "{q}", "category": "{c}", "subcategory": "{s}"}}')
    return "\n".join(results)

In [19]:
import openai
import os
import json

# ✅ ตั้งค่า OpenRouter
openai.api_key = ""  # ใส่key api
openai.api_base = "https://openrouter.ai/api/v1"

def generate_llm_prompt(description: str, image_analysis_results: list = None) -> str:
    context = retrieve_similar_context(description, top_k=3)

    image_info = ""
    if image_analysis_results:
        image_info = "\n**ข้อมูลการตรวจจับจากภาพ (กรุณาพิจารณาข้อมูลนี้อย่างละเอียด):**\n" # เน้นและเพิ่มคำสั่ง
        for prediction in image_analysis_results:
            # ปรับรูปแบบให้ชัดเจนขึ้น
            image_info += f'- **Class:** {prediction.get("class", "N/A")}, **Confidence:** {prediction.get("confidence", "N/A"):.2f}, **Class ID:** {prediction.get("class_id", "N/A")}\n'


    return f"""
คุณคือผู้ช่วยทันตแพทย์ AI วิเคราะห์อาการของผู้ป่วยด้านทันตกรรมโดยใช้ข้อมูลจริงและเหตุผลเชิงคลินิก

**อาการของผู้ป่วย:**
"{description}"

**ข้อมูลอ้างอิงจากคลินิกจริง:**
{context}
{image_info}

กรุณาจำแนกประเภทบริการที่ควรได้รับ พร้อมเหตุผลเชิงคลินิกที่อ้างอิงจากอาการของผู้ป่วยและข้อมูลการตรวจจับจากภาพ และระบุระดับความเร่งด่วน

ตอบกลับเป็น JSON เท่านั้น:
{{
  "predicted_category": "...",
  "predicted_subcategory": "...",
  "reasoning": "...",
}}
"""

def analyze_text_with_llm(prompt: str) -> dict:
    try:
        response = openai.ChatCompletion.create(
          model="openai/gpt-4.1-mini",
            messages=[
                {"role": "system", "content": "คุณคือผู้ช่วยทันตแพทย์ AI"},
                {"role": "user", "content": prompt}
            ]
        )
        print(response)
        content = response["choices"][0]["message"]["content"]

        # ตรวจสอบว่า content เป็น JSON ที่ถูกต้องหรือไม่ก่อน parse
        try:
            return json.loads(content.strip())
        except json.JSONDecodeError:
            print("Warning: LLM response was not valid JSON.")
            # ถ้าไม่ใช่ JSON ที่ถูกต้อง ให้ส่งคืนค่า default หรือจัดการตามเหมาะสม
            return {
                "predicted_category": "Unknown",
                "predicted_subcategory": "Unknown",
                "reasoning": "LLM did not return valid JSON. Could not analyze text."
            }

    except Exception as e:
        print("LLM Error:", e)
        return {}

# ตัวอย่างการเรียกใช้ (ถ้าต้องการทดสอบ)
# image_results_example = [{"class": "CALCULUS", "confidence": 0.95, "class_id": 0}]
# prompt = generate_llm_prompt("ฟันไม่สวย มีหินปูนและเจ็บเหมือกมีอาการเหงือกแดงบวมร่วมด้วย", image_results_example)
# analyze_text_with_llm(prompt)

In [16]:

from inference_sdk import InferenceHTTPClient

roboflow_client = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key=""  # ใส่ api key
)

def analyze_image_with_roboflow(image_path: str):
    result = roboflow_client.infer(image_path, model_id="dent_final2-hqfas/3")
    return result.get("predictions", [])


In [17]:
def merge_results(text_result, image_result):
    image_classes = [p["class"] for p in image_result]
    max_conf = max([p["confidence"] for p in image_result], default=0.0)

    category = text_result.get("predicted_category", "")
    subcat = text_result.get("predicted_subcategory", "")
    reasoning = text_result.get("reasoning", "")

    return {
        "category": category,
        "subcategory": subcat,
        "reasoning": reasoning,
        "image_findings": image_result,
        "image_max_confidence": max_conf
    }


In [18]:
import gradio as gr
import uuid
import cv2 # เพิ่มไลบรารี OpenCV สำหรับประมวลผลภาพ
import numpy as np

def process_dental_assistant(symptoms, image):
    llm_input_description = ""
    image_result = []
    output_image_path = None

    # 🖼️ วิเคราะห์ภาพด้วย Roboflow ถ้ามีภาพ
    if image is not None:
        image_path = image
        image_result = analyze_image_with_roboflow(image_path)
        # 🎨 วาด bounding boxes บนรูปภาพ
        img = cv2.imread(image_path)
        if img is not None:
            for prediction in image_result:
                x = int(prediction['x'])
                y = int(prediction['y'])
                width = int(prediction['width'])
                height = int(prediction['height'])
                class_name = prediction['class']
                confidence = prediction['confidence']

                # คำนวณ coordinates ของ bounding box (OpenCV ใช้ top-left corner และ bottom-right corner)
                x1 = int(x - width/2)
                y1 = int(y - height/2)
                x2 = int(x + width/2)
                y2 = int(y + height/2)

                # วาดสี่เหลี่ยม
                color = (0, 255, 0) # สีเขียว (BGR)
                thickness = 2
                cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)

                # ใส่ label
                label = f"{class_name}: {confidence:.2f}"
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.5
                font_thickness = 1
                text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
                text_x = x1
                text_y = y1 - 10 if y1 - 10 > 10 else y1 + text_size[1] + 10 # ปรับตำแหน่ง label

                cv2.putText(img, label, (text_x, text_y), font, font_scale, color, font_thickness, cv2.LINE_AA)

            # บันทึกรูปภาพที่แก้ไขแล้วชั่วคราว
            output_image_path = f"/tmp/{uuid.uuid4()}.png"
            cv2.imwrite(output_image_path, img)
        else:
            print(f"Error loading image: {image_path}")


    # ➡️ สร้างคำอธิบายอาการสำหรับ LLM
    if symptoms and symptoms.strip(): # ถ้ามี input symptoms จากผู้ใช้ ให้ใช้ค่านี้
        llm_input_description = symptoms
        print(f"ใช้คำอธิบายจากผู้ใช้: {llm_input_description}")
    elif image_result: # ถ้าไม่มี input symptoms จากผู้ใช้ แต่มีผลลัพธ์จากภาพ ให้ใช้ผลลัพธ์จากภาพ
        image_classes = [p["class"] for p in image_result]
        llm_input_description = "ตรวจพบ: " + ", ".join(image_classes)
        print(f"ใช้คำอธิบายจากภาพ: {llm_input_description}")
    else: # ถ้าไม่มีทั้งข้อความและภาพ ให้แจ้งว่าไม่มีข้อมูล
        llm_input_description = "ไม่มีข้อมูลอาการหรือภาพ"
        print(f"ไม่มีข้อมูลอินพุต")
        return {"category": "N/A", "subcategory": "N/A", "reasoning": "ไม่มีข้อมูลอินพุต", "image_findings": [], "image_max_confidence": 0.0}, None


    # 🔍 วิเคราะห์ข้อความด้วย LLM (ใช้คำอธิบายที่เลือกและผลวิเคราะห์ภาพ)
    prompt = generate_llm_prompt(llm_input_description, image_result) # ส่ง image_result ไปด้วย
    text_result = analyze_text_with_llm(prompt)
    print(text_result)

    # ↤️ รวมผล
    merged = merge_results(text_result, image_result)

    # ✅ ปรับรูปแบบ output เมื่อไม่มีภาพ
    if image is None:
        return merged.get("reasoning", "ไม่สามารถวิเคราะห์ได้เนื่องจากไม่มีข้อมูลภาพ"), None # ส่งแค่ reasoning และไม่มีรูปภาพ
    else:
        # ➡️ ส่งกลับผลลัพธ์ LLM และ path ของรูปภาพที่มีการวาด
        return merged, output_image_path

# ✅ สร้าง GUI
gr.Interface(
    fn=process_dental_assistant,
    inputs=[
        gr.Textbox(label="อธิบายอาการที่คุณพบ (หรือไม่ก็ได้)", lines=4, elem_id="symptoms-input"), # เพิ่ม elem_id
        gr.Image(type="filepath", label="แนบภาพฟัน (JPG/PNG)") # ทำให้ image ไม่ require
    ],
    outputs=[
        gr.JSON(label="ผลวิเคราะห์โดย AI (ข้อความ)", elem_id="analysis-output"), # เพิ่ม elem_id
        gr.Image(label="ผลวิเคราะห์โดย AI (ภาพ)") # เพิ่ม output สำหรับรูปภาพที่มีการวาด
    ],
    title="🦷 AI Dental Assistant with LLM + Roboflow + RAG", # ปรับชื่อ title
    css="""
    #symptoms-input textarea {
        font-size: 1.2em !important; /* Adjust the size as needed */
    }
    #analysis-output .json-viewer {
        font-size: 1.2em !important; /* Adjust the size as needed */
    }
    """ # เพิ่ม CSS สำหรับปรับขนาดตัวอักษรของ input และ output json
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f44d438c638d845d84.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
